In [2]:
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import cryptography.exceptions
from cryptography.hazmat.primitives import padding


In [4]:
class Receiver:
    def __init__(self, password, n):
        self.password = password
        self.shared_key = None
        self.words = None
        self.salt = None
        self.n = n

    def verify_Auth(self, message, signature):
        h = hmac.HMAC(self.words, hashes.SHA256())
        h.update(message)
        try: 
            h.verify(signature)
            return True
        except cryptography.exceptions.InvalidSignature:
            return False

    def decode (self, ciphertext):
        plaintext = b''
        #aplica a mesma função de encode ao ciphertext
        for i in range (0,len(ciphertext),8):
            p = ciphertext[i:i+8]
            for index, block in enumerate(p):   
                plaintext += bytes([block ^ self.words[i*8:(i+1)*8][index]])   
        #faz o unpadded para remover os caracteres adicionados 
        unpadder = padding.PKCS7(64).unpadder()
        unpadded = unpadder.update(plaintext) + unpadder.finalize()
        return unpadded.decode('utf-8')


    def get_seed(self, password):
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=self.salt,
            iterations=390000,)
        seed = kdf.derive(bytes(password, 'utf-8'))
        return seed


    def PRG(self, seed, n):
        #64 bites = 8 blocos de 1 byte(8 bits)
        #gerador com limite de 2^n paralavras 
        dig = hashes.Hash(hashes.SHAKE256(8 * pow(2,n)))
        dig.update(seed)
        rand = dig.finalize()
        self.words = rand


    def divide_data(self, data):
        # auth + self.salt + ciphertext
        # 0 a 31 - signature : 32 bytes
        # 32 a 47 - salt : 16 bytes
        # a partir do 48  - texto cifrado
        sign = data[:32]
        self.salt = data[32:48]
        texto_cifrado = data[48:] 
        return sign, texto_cifrado


    def readMessage(self, data):
        #interpreta a mensagem recebida
        sign, texto_cifrado = self.divide_data(data)
        #Gera as palavras para decifrar a mensagem
        self.PRG(self.get_seed(self.password), self.n)    
        #verifica a autenticidade da mensagem
        if(self.verify_Auth(b'This is my signature', sign)):
            plaintext = self.decode(texto_cifrado)
            print("Texto decifrado:")
            print(plaintext)
        else:
            raise Exception("Autenticidade tem falhas!")
       
        